In this assignment notebook, you will perform the preprocessing of the dataset "data-science-bootcamp/content/datasets/classification/raw/classification_25.csv". 

All the cells are filled with commented instructions and hints and partial codes. The parts that you are supposed to write your code is indicated with three dots "...".

You can find the solution to the assignment in the notebook "data-science-bootcamp/content/04_data_preprocessing_&_feature_engineering/Classification_dataset_preprocessing.ipynb"

The dataset is a modified version of "default_of_credit_card_clients.xls" from https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients. The slight modifications are done for the pedagogical reasons of demonstrating different methods of preprocessing and feature engineering (particularly, missing value imputation and categorical encoding).
This binary classification dataset contains 23 different attributes of the customers and 1 binary column "default payment next month" which is the target column to be predicted by the ML model. 

The notebook contains the following sections:

0. Exploratory Data Analysis
1. Missing Value Imputation
1.0. Identifying columns with null values \
1.1. Complete Case Analysis\
1.2. Mean or Median Imputation \
1.3. Frequent Category Imputation (Mode Imputation)
2. Categorical encoding\
2.1. One Hot Encoding 
3. Handling Outliers\
3.1. Identifying the outliers \
3.2. Outlier trimming \
3.3. Outlier Capping
4. Feature Scaling\
4.1. Min-Max Scaling
5. Feature Selection\
5.1. Dropping Constant Features \
5.2. Correlation-based Feature Selection\
5.2.a Selecting Features with hight correlations with Target\
5.2.a Selecting Features with low correlations with other features
6. Saving the preprocessed data

# 0. Exploratory Data Analysis (EDA)

In [ ]:
#import the necessary libraries (pandas, numpy, matplotlib.pyplot, seaborn) 
import warnings
warnings.filterwarnings('ignore')
...


### 0.1 Loading the data into a dataframe

In [ ]:
# load the "../datasets/classification/raw/classification_25.csv" into a dataframe called "df"
# note that the delimiter is ";"
df = ...

In [ ]:
#display the first 5 rows of df to familiarize yourself with the data
df.head()

In [ ]:
# drop the first column ("ID")
# do not forget to set inplace= True
...

In [ ]:
# check the shape of the dataframe as a 2-dimensional data object (#rows, #columns)
df.shape

The dataset contains 30000 records (rows) and 24 columns (features + target):

The target column (y) is called "default payment next month".
The rest 23 columns (X) are our feature vectors.

In [ ]:
# store the "default payment next month" in a variable called "target", and the rest of columns' names in a variable called "X"
target = "default payment next month"
X = list(set(df.columns) - set([target])) 

In [ ]:
# plot the distribution of the classes in the target variable
...

In [ ]:
# calculate the imbalance ration of the target classes
class_0 = df[target].value_counts().to_list()[0]
class_1 = df[target].value_counts().to_list()[1]
original_imbalance_ratio = class_1/class_0
print("Imbalance ratio =", original_imbalance_ratio)

The imbalance ratio is 0.28 which is good enough for this classification problem, given the size of the dataset (30000)\
we will later compare the original_imbalance_ratio with the train_imbalance_ration which is obtained after train/test split and demand that they are not very different.

In [ ]:
# We first have a look at the data types of all columns
...

In [ ]:
# classifying feature columns (X) with different data types:
num_cols = []
for col, dtype in df[X].dtypes.to_dict().items():
    if str(dtype) in 'int64' or str(dtype) in 'float64':
        num_cols.append(col)
print("Number of numerical Columns:", len(num_cols))

str_cols = ...

print("Number of String Columns:", len(str_cols))

# 1. Missing Value Imputation

### 1.0. Identifying columns with null values

In [ ]:
# create a list, called "null_cols", of all columns with null values
null_cols = ...

In [ ]:
print(null_cols)

In [ ]:
print("Number of Columns with Null values:", len(null_cols))
print("Percentage of null values:")
for col in null_cols:
    print(col, round(df[col].isnull().mean(), 4), ", Data Type: ", df[col].dtype)

We observe that "EDUCATION" and "AGE" have less than 2% missing values and "MARRIAGE" has 0.18% missing value.\
For the column "MARRIAGE", we perform Complete Case Analysis,\
For the integer column "AGE", we perform mean/median imputation, \
For the categorical column "EDUCATION", we perform Frequent Category Imputation.

### 1.1. Complete Case Analysis

In [ ]:
# for the column "MARRIAGE" drop all records which contain a null value
df = ...

In [ ]:
df.shape

We observe that after dropping the missing values, the number of records is reduced from 30000 to 29946.

### 1.2. Mean or Median Imputation: fillna()

Before filling the null values, we split the data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, df[target], test_size=0.3, random_state=10)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train_class_0 = y_train.value_counts().to_list()[0]
y_train_class_1 = y_train.value_counts().to_list()[1]
train_imbalance_ration = y_train_class_1/y_train_class_0
print("Original imbalance ratio =", original_imbalance_ratio)
print("Imbalance ratio after train/test split =", train_imbalance_ration)

We observe that train/test splitting procedure has a very tiny influence on the imbalance ration, which is what we want.

Before deciding whether we implement the mean or median imputation, we plot the distribution of "AGE" 

In [ ]:
# plot the distribution of df['AGE'] 
...

In [ ]:
print("Mean of AGE: ", df['AGE'].mean())
print("Median of AGE: ", df['AGE'].median())
print("Mode of AGE: ", df['AGE'].mode())

Since the data distribution is skewed, we perform median imputation

In [ ]:
# store the median of X_train['AGE'] in a new variable called "AGE_median" 
AGE_median = ...
# fill the missing values of X_train['AGE'] with AGE_median in a new column called "X_train['AGE_median']"
X_train['AGE_median'] = ...
# fill the missing values of X_test['AGE'] with AGE_median in a new column called "X_test['AGE_median']"
X_test['AGE_median'] = ...

In [ ]:
# plot the distribution of X_train['AGE'] and X_train['AGE_median'] with two different colors in one diagram
...

We observe that the distribution of the "AGE" column is not distorted by meadian imputation, as desired. Therefore, we keep the new column 'AGE_median' and drop the old column "AGE".

In [ ]:
# drop the column "AGE" from X_train and X_test
X_train.drop(columns=["AGE"], axis=1, inplace=True)
X_test.drop(columns=["AGE"], axis=1, inplace=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 1.3. Frequent Category Imputation (Mode Imputation)

In [ ]:
# plot the distribution of values in X_train['EDUCATION']
...

In [ ]:
# store the mode of X_train["EDUCATION"] in a variable called "EDUCATION_MODE"
EDUCATION_MODE = ...

The most frequent catregories in X_train['EDUCATION'] is the "university" category.

In [ ]:
# implement the mode imputation for X_train['EDUCATION'] and X_test['EDUCATION']
...

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 2. Categorical encoding

In [ ]:
# Recall that we have 3 categorical columns
print(str_cols)

In [ ]:
# print the distinct catgories in the categorical columns
for col in str_cols:
    print(col, X_train[col].unique())

### 2.1. One Hot Encoding (using Scikit-Learn):
We encode the three categorical features using the OHE.

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# import the OneHotEncoder from sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
# create an encoder as an instance of OneHotEncoder with parameters sparse= False, handle_unknown='ignore'
encoder = ...
# fit the encoder to the X_train[str_cols]


In [ ]:
# transform the categorical columns in X_train
X_train_transformed = encoder.transform(X_train[str_cols])
# turn the transformed Numpy array into a Pandas dataframe
X_train_OHE_skl = pd.DataFrame(X_train_transformed)

In [ ]:
X_train_OHE_skl

In [ ]:
# as we can see, the new encoded columns are named as integers. 
# However, the names of the features are stored in the get_feature_names() method
encoder.get_feature_names()

In [ ]:
# create a dummy_cols_list which contains the names of the categorical columns and their category in the format "col_cat"
dummy_cols_list = list(encoder.get_feature_names())
for i in range(0, len(dummy_cols_list)):
    for j in range(0, len(str_cols)):
        dummy_cols_list[i] = dummy_cols_list[i].replace(f"x{j}_", str_cols[j] + "_") 

In [ ]:
# we rename the dataframe columns to the above names stored in dummy_cols_list
X_train_OHE_skl.columns = ...

In [ ]:
X_train_OHE_skl.head()

In [ ]:
# transform the categorical columns in X_test
X_test_transformed = ...
# turn the transformed Numpy array into a Pandas dataframe
X_test_OHE_skl = ...

In [ ]:
# rename the new encoded columns using dummy_cols_list
X_test_OHE_skl.columns = ...

In [ ]:
X_train_OHE_skl.shape, X_test_OHE_skl.shape

We now concatinate the original X_train/X_test and the encoded X_train_OHE_skl/X_test_OHE_skl (and then drop the categorical columns from the resulting dataframe). 

In [ ]:
X_train = pd.concat([X_train.reset_index(drop=True), X_train_OHE_skl.reset_index(drop=True)], axis=1)
X_train = X_train.drop(columns=str_cols, axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_OHE_skl.reset_index(drop=True)], axis=1)
X_test = X_test.drop(columns=str_cols, axis=1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 3. Handling Outliers
3.1. Identifying the outliers \
3.2. Outlier trimming \
3.3. Outlier Capping


### 3.1. Identifying the outliers

In [ ]:
# plot the box-plot of feature columns (it is sufficient to plot only a couple of them ['PAY_AMT2', 'BILL_AMT1', 'BILL_AMT3', 'PAY_AMT1', 'PAY_AMT3'])
plt.rcParams["figure.figsize"] = (50,10)
fig, axs = plt.subplots(ncols=5, constrained_layout=True)
sns.boxplot(y=..., ax=axs[0])
sns.boxplot(y=..., ax=axs[1])
sns.boxplot(y=..., ax=axs[2])
sns.boxplot(y=..., ax=axs[3])
sns.boxplot(y=..., ax=axs[4])
plt.show()


In [ ]:
# calculate the IQR, lower_boundary, upper_boundary and outlier_list of all columns.
# create the following four dictionaries and store the relevant data in them
IQR_dict= {}
for col in X_train.columns:
    IQR_dict[f"{col}_IQR"] = ...

lower_boundary_dict= {}
for col in X_train.columns:
    lower_boundary_dict[f"{col}_lower_boundary"] = ...

upper_boundary_dict= {}
for col in X_train.columns:
    upper_boundary_dict[f"{col}_upper_boundary"] = ...

columns_outliers_dict={}
for col in X_train.columns:
    columns_outliers_dict[f"{col}_outliers_list"] = ...

In [ ]:
print(" >>IQR_dict: ", IQR_dict)
print(" >>lower_boundary_dict: ", lower_boundary_dict)
print(" >>upper_boundary_dict: ", upper_boundary_dict)

In [ ]:
# creatre a list, called "outlier_cols", containing all columns' names which have more than 0% outliers.
outlier_cols =...
print(outlier_cols)

### 3.2. Outlier trimming
Drop all the records which include an outlier (in X_train)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
#create a list, called "outlier_10percent_cols", of columns with less than 10% outliers
outlier_10percent_cols = [col for col in outlier_cols if columns_outliers_dict[f"{col}_outliers_list"].mean() < 0.1 ] 

print(outlier_10percent_cols)

In [ ]:
# drop all the records of the outlier_10percent_cols which contain outliers in X_train and y_train
X_train = X_train.loc[~(
+ columns_outliers_dict[...]
+ columns_outliers_dict[...]
), 
]

y_train = y_train.loc[~(
+ columns_outliers_dict[...]
+ columns_outliers_dict[...]
), 
]

In [ ]:
X_train.shape , X_test.shape, y_train.shape, y_test.shape

The outlier trimming procedure has reduced the 20962 records in X_train/y_train to 20659 records.\
We also need to check if the Imbalance ration is changed.

In [ ]:
y_train_class_0 = y_train.value_counts().to_list()[0]
y_train_class_1 = y_train.value_counts().to_list()[1]
print("Imbalance ratio X_train after outlier trimming =", y_train_class_1/y_train_class_0)
print("Original imbalance ratio =", original_imbalance_ratio)

We observe that the outlier trimming procedure has a very tiny influence on the imbalance ration, which is what we desired.

### 3.3. Outlier Capping

In [ ]:
# create a set of the columns with more than 10 percent outliers
outlier_90percent_cols = ...

In [ ]:
print(outlier_90percent_cols)

In [ ]:
# implement the outlier capping method for outlier_90percent_cols
for col in outlier_90percent_cols:
    X_train[col] = ...

In [ ]:
X_train.shape , X_test.shape, y_train.shape, y_test.shape

We observe that outlier capping does not reduce the dataset size.

# 4. Feature Scaling

### 4.1. Min-Max Scaling
we perform min-max scaling using sklearn

In [ ]:
# import MinMaxScaler from sklearn.preprocessing
...
# create a scaler as an instance of MinMaxScaler
scaler = ...
# fit X_train to the scaler
...
# transform X_train and X_test using the fitted scaler
X_train_scaled = ...
X_test_scaled = ...

# 5. Feature Selection
5.1. Dropping Constant Features \
5.2. Correlation-based Feature Selection\
5.2.a Selecting Features with low correlations with other features\
5.2.a Selecting Features with hight correlations with Target

### 5.1. Dropping Constant Features

In [ ]:
# create a list of constant features in X_train (i.e. those whose number of unique values is 1)
const_feat = ...
print(const_feat)

In [ ]:
# drop the constant features from X_train and X_test
...
...

In [ ]:
X_train.shape , X_test.shape, y_train.shape, y_test.shape

### 5.2. Correlation-based Feature Selection

### 5.2.a Selecting Features with hight correlations with Target

In [ ]:
# We first add the target (y_train) to the Features (X_train) to be able to calculate the correlation matrix between them
df_train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis = 1)
# create the correlation matrix
cor_mat = ...
# Add a column which contains the sorted absolute values of corr(X_train, y_train)
cor_mat["X-Y_corr"]= ...

In [ ]:
cor_mat["X-Y_corr"]

In [ ]:
# collect all the features with corr(X_train, y_train) > 0.014 into a list
final_cols = cor_mat[cor_mat["X-Y_corr"] > 0.014].index.values.tolist()
final_cols = list(set(final_cols) - set([target]))

In [ ]:
len(final_cols)

In [ ]:
# replace the columns in X_train and X_test with final_cols
X_train = ...
X_test = ...

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

### 5.2.b Selecting Features with low correlations with other features

In [ ]:
# plot the correlation diagram of X_train
...

In [ ]:
# create a set of column pairs whose correlation is higher than 88%
col_corr = set()
corr_matrix = X_train.corr()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.9: 
            print(abs(corr_matrix.iloc[i, j]), corr_matrix.columns[i], corr_matrix.columns[j])
            colname = corr_matrix.columns[j]
            col_corr.add(colname)

In [ ]:
# dropping one of the columns in each pair in col_corr in X_train and X_test
...
...

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# 6. Saving the preprocessed data

In [ ]:
# save the preprocessed data (X_train, y_train, X_test, y_test) as four csv files with "," as delimiter, and with "utf-8" encoding to the folder "../datasets/classification/processed"
X_train.to_csv(...)
X_test.to_csv(...)
y_train.to_csv(...)
y_test.to_csv(...)